In [ ]:
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
import seaborn as sns
from scipy.stats import chi2_contingency

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('data/data_final_preprocessed.csv')

In [ ]:
contingency_table = pd.crosstab(df['pro_flg'], df['success_flg'])
print("Contingency Table:")
print(contingency_table)

chi2, p, dof, expected = chi2_contingency(contingency_table, correction=True)

print(f"X^2 Statistic: {chi2}")
print(f"P-value: {p}")

In [ ]:
df = pd.read_csv('data\data_final_preprocessed.csv')
for col in df['gender_cd'].unique():
    df1 = df[df['gender_cd'] == col]
    X = df1[['pro_flg']]
    y = df1['success_flg']
    X = sm.add_constant(X)
    print(f'----------------------------{col}----------------------------')
    model = sm.Logit(y, X)
    model_result = model.fit()
    print(model_result.summary())

In [ ]:
# Другие каналы
# для не про больше звонить
# для про другой канал/не ходить
# оптимизация работы колл центра
# на полписку сильных
# на обычных послабее

X = df[['pro_flg', 'gender_cd']]
X = sm.add_constant(X)
y = df['success_flg']

model = sm.Logit(y, X)
model_result = model.fit()

print(model_result.summary())

Проверка на устойчивость по хи квадрату

In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency

df1 = df.dropna(subset=['gender_cd'])

for col in df1['gender_cd'].unique():
    print(f"Processing = {col}")

    df_filtered = df1[df1['gender_cd'] == col]

    # Проверяем, что датафрейм не пустой
    if df_filtered.empty:
        print(f"No data for = {col}. Skipping...")
        continue

    # Проверяем, что есть как минимум две категории в 'pro_flg' и 'success_flg'
    if len(df_filtered['pro_flg'].unique()) < 2 or len(df_filtered['success_flg'].unique()) < 2:
        print(f"Not enough categories for = {col}. Skipping...")
        continue

    # Создаем таблицу сопряженности
    contingency_table = pd.crosstab(df_filtered['pro_flg'], df_filtered['success_flg'])
    print("Contingency Table:")
    print(contingency_table)

    # Выполняем хи-квадрат тест
    chi2, p, dof, expected = chi2_contingency(contingency_table)

    # Вывод результатов хи-квадрат теста
    print("\nChi-Square Test Results:")
    print(f"Chi-Square Statistic: {chi2}")
    print(f"P-value: {p}")
    print("-" * 50)

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(df['success_flg'], model_result.predict())

In [ ]:
# список для хранения результатов тестирования
results = []
df = pd.read_csv('data\data_final_preprocessed.csv')

# Группировка данных по продуктам
for product, group in df.groupby('product'):
    contingency_table = pd.crosstab(group['bundle_nm'], group['success_flg'])

    # Проверка, достаточно ли данных для проведения теста
    if contingency_table.shape[0] < 2 or contingency_table.shape[1] < 2:
        # Если данных недостаточно, добавляем результат с указанием на отсутствие данных
        results.append({
            'product': product,
            'test': 'Нет данных',
            'p_value': None,
            'significant': None
        })
        continue

    # Проверка на минимальное значение в контингентной таблице
    if contingency_table.values.min() < 5:
        # Если минимальное значение меньше 5, используем тест Фишера
        stat, p_value = stats.fisher_exact(contingency_table, alternative='two-sided')
        test_type = 'Fisher'  # Указываем тип теста
    else:
        # В противном случае используем критерий хи-квадрат
        stat, p_value, _, _ = stats.chi2_contingency(contingency_table)
        test_type = 'Chi2'  # Указываем тип теста

    # Добавление результатов теста в список
    results.append({
        'product': product,
        'test': test_type,
        'p_value': p_value,
        'significant': p_value < 0.05  # Определяем, является ли результат статистически значимым
    })

results_df = pd.DataFrame(results)

results_df


In [ ]:
df[(df['success_flg'] == 1) & (df['functional_cd'] == 'Utilization')].wo_hit_status_reason_desc.value_counts()

In [ ]:
df.groupby(['product', 'pro_flg'])['confirmed_income'].agg(mean = 'mean', count = 'count')

In [ ]:
df = pd.read_csv('data\data_final_preprocessed.csv')
df = df[df['function'] == 'Продажи и предложения']
for col in df['product'].unique():
    df1 = df[df['product'] == col]
    X = df1[['pro_flg', 'age', 'state_talk_time_sec']]
    y = df1['success_flg']
    X = sm.add_constant(X)
    print(f'----------------------------{col}----------------------------')
    model = sm.Logit(y, X)
    model_result = model.fit()
    print(model_result.summary())

In [ ]:
df = pd.read_csv('data\data_final_preprocessed.csv')
X = df[['pro_flg', 'state_talk_time_sec', 'age']]
y = df['success_flg']
X = sm.add_constant(X)

model = sm.Logit(y, X)
model_result = model.fit()

print(model_result.summary())

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(df['success_flg'], model_result.predict())

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns

vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]

vif_data

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Get predicted probabilities from the probit model
y_pred_prob = model_result.predict(X)

# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(y, y_pred_prob)

# Calculate AUC
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(6, 4))
sns.set_style("whitegrid")  # Настройка стиля фона

# Устанавливаем цвет фона
plt.gcf().set_facecolor('#F6F6F6')
plt.plot(fpr, tpr, color='#864BA5', lw=2, label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()


In [ ]:
df = pd.read_csv('data\data_final_preprocessed.csv')
values = df.region_size.value_counts().reset_index()
values

In [ ]:
plt.figure(figsize=(16, 8))

sns.set_style("whitegrid")
plt.gcf().set_facecolor('#F6F6F6')

sns.barplot(data=values, x='region_size', y='count', palette=['#C89AFF', '#C89AFF', '#864BA5', '#864BA5'])
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.show()

In [ ]:
plt.figure(figsize=(8, 3))
sns.set_style("whitegrid")  # Настройка стиля фона

names = ['const', 'pro_flg', 'age', 'state_talk_time_sec']
coefs = [-2.95, -0.29, -0.12, 1.12]

# Устанавливаем цвет фона
plt.gcf().set_facecolor('#F6F6F6')
sns.barplot(x=names, y=coefs, color='#864BA5')

In [ ]:
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, accuracy_score
import matplotlib.pyplot as plt

df = pd.read_csv('data/data_final_preprocessed.csv')

X = df[['pro_flg', 'age', 'state_talk_time_sec']]
y = df['success_flg']

X = sm.add_constant(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = sm.Logit(y_train, X_train)

model_result = model.fit()

print(model_result.summary())

y_pred_prob = model_result.predict(X_test)

threshold = 0.5
y_pred = (y_pred_prob >= threshold).astype(int)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

print(f"AUC: {roc_auc:.4f}")


In [ ]:
df = pd.read_csv('data\data_final_preprocessed.csv')
df.dropna(subset=['region_size'], inplace=True)
for col in df['region_size'].unique():
    df1 = df[df['region_size'] == col]
    X = df1[['pro_flg', 'age', 'state_talk_time_sec']]
    y = df1['success_flg']
    X = sm.add_constant(X)
    print(f'---------------------------------------------------------------------------------------')
    print(f'-----------------------------------------{col}-----------------------------------------')
    print(f'---------------------------------------------------------------------------------------')
    model = sm.Logit(y, X)
    model_result = model.fit()
    print(model_result.summary())

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.genmod import families
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.interpolate import interp1d

# Загрузка данных
df = pd.read_csv('data/data_final_preprocessed.csv')

# Создаем словарь для хранения результатов по каждому продукту
roc_data = {}

df.dropna(subset=['region_size'], inplace=True)

# Проходим по каждому уникальному продукту
for col in df['region_size'].unique():
    df1 = df[df['region_size'] == col]
    X = df1[['pro_flg', 'age', 'state_talk_time_sec']]
    y = df1['success_flg']
    X = sm.add_constant(X)

    print(f'----------------------------{col}----------------------------')
    model = sm.Logit(y, X)
    model_result = model.fit()
    print(model_result.summary())

    # Получаем предсказанные вероятности
    y_pred_prob = model_result.predict(X)

    # Вычисляем ROC-кривую и AUC
    fpr, tpr, thresholds = roc_curve(y, y_pred_prob)
    roc_auc = auc(fpr, tpr)

    # Сохраняем результаты для текущего продукта
    roc_data[col] = {'fpr': fpr, 'tpr': tpr, 'auc': roc_auc}

# Построение ROC-кривых для всех продуктов на одном графике
plt.figure(figsize=(10, 7))
sns.set_style("whitegrid")  # Настройка стиля фона

# Устанавливаем цвет фона
plt.gcf().set_facecolor('#F6F6F6')

# Проходим по всем продуктам и строим их ROC-кривые
for product, data in roc_data.items():
    # Интерполяция для сглаживания кривой
    fpr_smooth = np.linspace(0, 1, 100)
    tpr_smooth = interp1d(data['fpr'], data['tpr'], kind='linear')(fpr_smooth)

    plt.plot(fpr_smooth, tpr_smooth, label=f'{product} (AUC = {data["auc"]:.2f})', lw=2)

# Добавляем диагональную линию (случайное угадывание)
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')

# Настройка графика
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve for Each Product')
plt.legend(loc='lower right')
plt.show()